In [ ]:
from GoogleNews import GoogleNews
import dateparser

googlenews = GoogleNews(start='01/01/2021', end='12/30/2025')
googlenews.search('Tata Motors')

#Get first 5 pages (or more)
for page in range(1, 6):
    googlenews.getpage(page)
    results = googlenews.result()
    for r in results:
        print(r['date'], r['title'])


HTTP Error 429: Too Many Requests


In [14]:
from newspaper import Article
import pandas as pd

# Example list of Tata Motors news URLs (you need to collect them first)
urls = [
    'https://www.moneycontrol.com/india/stockpricequote/auto-lcvshcvs/tatamotors/TM03'
    # 'https://economictimes.indiatimes.com/industry/auto/auto-new/tata-motors-to-demerger-into-ev-and-commercial-vehicle-businesses/articleshow/107357159.cms',
    # Add more URLs here
]

articles_data = []

for url in urls:
    try:
        article = Article(url)
        article.download()
        article.parse()

        articles_data.append({
            'date': article.publish_date.strftime('%Y-%m-%d') if article.publish_date else 'Unknown',
            'title': article.title,
            'text': article.text,
            'link': url
        })
    except Exception as e:
        print(f'Failed to process {url}: {e}')

# Convert to DataFrame
df = pd.DataFrame(articles_data)
print(df.head())


      date                                              title  \
0  Unknown  Tata Motors Share Price, Tata Motors Stock Pri...   

                                                text  \
0  Disclaimer\n\nThe Moneycontrol Stock Score is ...   

                                                link  
0  https://www.moneycontrol.com/india/stockpriceq...  


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

options = Options()
# options.add_argument('--headless')
driver = webdriver.Chrome(service=Service(), options=options)

driver.get("https://www.bseindia.com/corporates/ann.html")
driver.maximize_window()

# Wait for the search box to appear
wait = WebDriverWait(driver, 10)
search_box = wait.until(EC.presence_of_element_located((By.ID, "getquotesearch")))

search_box.send_keys("TATA MOTORS LTD.")  # Full name in uppercase as per BSE
search_box.send_keys(Keys.ENTER)
time.sleep(2)
search_box.send_keys(Keys.ENTER)


# Click Submit
driver.find_element(By.ID, "ContentPlaceHolder1_btnSubmit").click()
time.sleep(3)

# Collect data
data = []
rows = driver.find_elements(By.CSS_SELECTOR, "table#ContentPlaceHolder1_gvData tr")

for row in rows[1:]:
    cols = row.find_elements(By.TAG_NAME, "td")
    if len(cols) >= 4:
        date = cols[0].text
        headline = cols[1].text
        link = cols[1].find_element(By.TAG_NAME, "a").get_attribute("href")
        data.append({"Date": date, "Headline": headline, "Link": link})

driver.quit()

df = pd.DataFrame(data)
print(df)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="ContentPlaceHolder1_btnSubmit"]"}
  (Session info: chrome=138.0.7204.158); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
	GetHandleVerifier [0x0x7ff65e78e925+77845]
	GetHandleVerifier [0x0x7ff65e78e980+77936]
	(No symbol) [0x0x7ff65e549cda]
	(No symbol) [0x0x7ff65e5a06aa]
	(No symbol) [0x0x7ff65e5a095c]
	(No symbol) [0x0x7ff65e5f3d07]
	(No symbol) [0x0x7ff65e5c890f]
	(No symbol) [0x0x7ff65e5f0b07]
	(No symbol) [0x0x7ff65e5c86a3]
	(No symbol) [0x0x7ff65e591791]
	(No symbol) [0x0x7ff65e592523]
	GetHandleVerifier [0x0x7ff65ea6683d+3059501]
	GetHandleVerifier [0x0x7ff65ea60bfd+3035885]
	GetHandleVerifier [0x0x7ff65ea803f0+3164896]
	GetHandleVerifier [0x0x7ff65e7a8c2e+185118]
	GetHandleVerifier [0x0x7ff65e7b053f+216111]
	GetHandleVerifier [0x0x7ff65e7972d4+113092]
	Get

In [17]:
import feedparser
import pandas as pd

def get_bing_news(query, pages=1555):
    news_items = []
    for page in range(1, pages + 1):
        url = f"https://www.bing.com/news/search?q={query.replace(' ', '+')}&format=rss&first={page*10}"
        feed = feedparser.parse(url)
        
        for entry in feed.entries:
            news_items.append({
                'Date': entry.published,
                'Title': entry.title
            })
    return pd.DataFrame(news_items)

df = get_bing_news("Tata Motors", 1555)
print(df)


                                Date  \
0      Fri, 18 Jul 2025 02:33:00 GMT   
1      Fri, 18 Jul 2025 05:54:09 GMT   
2      Fri, 18 Jul 2025 06:39:10 GMT   
3      Fri, 18 Jul 2025 07:02:36 GMT   
4      Thu, 17 Jul 2025 22:42:00 GMT   
...                              ...   
15066  Mon, 02 May 2022 06:08:00 GMT   
15067  Wed, 13 Oct 2021 08:57:00 GMT   
15068  Mon, 28 Jun 2021 15:45:00 GMT   
15069  Mon, 28 Jun 2021 07:57:00 GMT   
15070  Thu, 16 Jun 2022 19:39:00 GMT   

                                                   Title  
0      Tata Group stocks: Tata Steel vs Trent vs Tita...  
1      Tata Motors approaches Italy's Agnelli family ...  
2      Tata Motors-owned JLR delays launch of Range R...  
3      Iveco Is Said to Draw Takeover Interest From I...  
4      Tata Punch Crosses 6 Lakh Sales Milestone In U...  
...                                                  ...  
15066  Tata Motors Ev Strategy: Tata, India's electri...  
15067  Tata Motors offers TPG multiple exit opt

In [89]:
df.tail()

,Date,Timex,Title
15066,"Mon, 02 May 2022",22:42:00,"Tata Motors Ev Strategy: Tata, India's electri..."
15067,"Wed, 13 Oct 2021",22:42:00,"Tata Motors offers TPG multiple exit options, ..."
15068,"Mon, 28 Jun 2021",22:42:00,Tata Motors plans 10 new electric vehicles by ...
15069,"Mon, 28 Jun 2021",22:42:00,Tatas plan 10 EVs by 2025 in push towards sust...
15070,"Thu, 16 Jun 2022",22:42:00,Tata Motors taking right actions to navigate c...


In [84]:
df['Date'] = df['Date'].str[:-13]

In [99]:
df['Date'] = df['Date'].str[-11:]

In [100]:
df

,Date,Title
0,18 Jul 2025,Tata Group stocks: Tata Steel vs Trent vs Tita...
1,18 Jul 2025,Tata Motors approaches Italy's Agnelli family ...
2,18 Jul 2025,Tata Motors-owned JLR delays launch of Range R...
3,18 Jul 2025,Iveco Is Said to Draw Takeover Interest From I...
4,17 Jul 2025,Tata Punch Crosses 6 Lakh Sales Milestone In U...
...,...,...
15066,02 May 2022,"Tata Motors Ev Strategy: Tata, India's electri..."
15067,13 Oct 2021,"Tata Motors offers TPG multiple exit options, ..."
15068,28 Jun 2021,Tata Motors plans 10 new electric vehicles by ...
15069,28 Jun 2021,Tatas plan 10 EVs by 2025 in push towards sust...
